# Global Sensitivity Attempt

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from deuterium import Variable, to_vec, random_symbols
import symengine as se
from sklearn.metrics import accuracy_score
import sys
from sympy import sympify
from scipy.optimize import shgo
sys.setrecursionlimit(1_000_000)
import warnings
warnings.filterwarnings("ignore")

Define some utility functions, notably the loss functions and tempered sigmoid activation functions.

In [3]:
to_data = np.vectorize(lambda x: x.data)

def sigmoid(x, s=1, T=1, o=0):
        return (s/(1+np.exp(-T*x)))-o

def tanh(x):
    return sigmoid(x, 2, 2, 1)

bce_loss = lambda y_pred, y_true: -np.mean(np.multiply(y_true, np.log(y_pred)) + np.multiply((1 - y_true), np.log(1 - y_pred)))
normalize = lambda x: (x-x.min())/(x.max()-x.min())

Define the network architecture

In [4]:
IN=25
INTERMEDIATE= 8

In [5]:
x = to_vec(np.array(random_symbols(IN, "x")).reshape((1,IN))) 
y = to_vec(np.array(random_symbols(1, "y")))

w1 = to_vec(np.array(random_symbols(IN*INTERMEDIATE, "w1")).reshape(IN, INTERMEDIATE))
b = to_vec(np.array(random_symbols(INTERMEDIATE, "b")).reshape(1, INTERMEDIATE))
w2 = to_vec(np.array(random_symbols(INTERMEDIATE, "w2")).reshape(INTERMEDIATE,1))

all_symbols = [symb for symb in np.concatenate((x.flatten(), y.flatten(), w1.flatten(), b.flatten(), w2.flatten()))]

In [6]:
#y_loss = (y_pred - y)**2

Symbolically calculate the network output

In [7]:
layer_1 = sigmoid(x@w1)+b
y_pred = sigmoid(layer_1@w2)
loss = bce_loss(y_pred, y)

In [8]:
loss.backward()

In [9]:
x_grad = np.array([i.grad for i in x.flatten().tolist()])
y_grad = np.array([i.grad for i in y.flatten().tolist()])
w1_grad = np.array([i.grad for i in w1.flatten().tolist()])
b_grad = np.array([i.grad for i in b.flatten().tolist()])
w2_grad = np.array([i.grad for i in w2.flatten().tolist()])

full_grad = to_vec(np.concatenate((x_grad, y_grad, w1_grad, b_grad, w2_grad)))

Obtain the global sensitivity term

s $\ge$ $||G($x_1$) - G($x_2$)||_2$ then 

$\forall$ $x_i$. s $\ge$ $||G($x_1$)||_2$ * 2

In [10]:
norm = np.linalg.norm(full_grad, ord=2)
#GS
gs = 2 * np.linalg.norm(full_grad, ord=2)  

# global_sens = se.Lambdify(to_data(all_symbols), gs, cse=True) #global sensitivities

SHGO - Finds the global minimum of a function using SHG optimization.



To obtain the Lipschitz constant of the query, we need to maximise the GS by minimising the negative of the GS using a suitable optimiser with a global optimality guarantee. 

In [11]:
norm_func = se.Lambdify(list(gs.data.free_symbols), - gs.data)

In [12]:
# # SHGO minimises the function given certain constraints, in this case the bounds on the variables.
# sol = shgo(norm_func, ((1,2), (0.5,3)))
# -sol.fun

Pre-compile all relevant functions to train using substitution later.

In [13]:
y_pred_func = se.Lambdify(to_data([symb for symb in np.concatenate((x.flatten(), w1.flatten(), b.flatten(), w2.flatten()))]), to_data(y_pred), cse=True) # output of net
loss_func = se.Lambdify(to_data(all_symbols), to_data(loss), cse=True) #loss
grad_func = se.Lambdify(to_data(all_symbols), (np.concatenate((w1_grad.flatten(), b_grad.flatten(), w2_grad.flatten()))), cse=True) # gradient w.r.t w+b only

full_grad_func = se.Lambdify(to_data(all_symbols), (np.concatenate((x_grad.flatten(), y_grad.flatten(), w1_grad.flatten(), b_grad.flatten(), w2_grad.flatten()))), cse=True) # gradient w.r.t w+b only

grad_norm = se.Lambdify(to_data(all_symbols), norm.data, cse=True) #gradient norm 
#global_sens = se.Lambdify(to_data(all_symbols), gs, cse=True) #global sensitivities

norm_func = se.Lambdify(list(gs.data.free_symbols), - gs.data) # for computing maximum with shgo
#partial_sens = se.Lambdify(to_data(all_symbols), N, cse=True) #partial sensitivities

## Substituting Real Values

The next functions generate synthetic data of vertical and horizontal bars with Gaussian noise.

In [14]:
def get_vertical(n=100, train=True):
    out = []
    vertical = np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],]).astype(np.float64).flatten()
    for _ in range(n):
        ar = vertical + np.random.normal(0, 0.2, size=vertical.shape)
        ar = normalize(ar)
        if train:
            out.append(np.concatenate((ar, [0.])))    
        else:
            out.append(ar)
    #print(np.array(out))
    return np.array(out)

def get_horizontal(n=100, train=True):
    out = []
    vertical = np.array([[0,0,0,0,0],[0,0,0,0,0],[1,1,1,1,1],[0,0,0,0,0],[0,0,0,0,0],]).astype(np.float64).flatten()
    for _ in range(n):
        ar = vertical + np.random.normal(0, 0.2, size=vertical.shape)
        ar = normalize(ar)
        if train:
            out.append(np.concatenate((ar, [1.])))
        else:
            out.append(ar)
    return np.array(out)

In [15]:
LR = 0.1

HOW_MANY = 1000

vertical = get_vertical(HOW_MANY)
horizontal = get_horizontal(HOW_MANY)

#weights = np.full((1, len(np.concatenate((w1.flatten(), b.flatten(), w2.flatten())))), 0.01)
weights = np.random.uniform(-1., 1., len(np.concatenate((w1.flatten(), b.flatten(), w2.flatten()))))[None, ...]
vert_batch = np.hstack((vertical, np.tile(weights, (HOW_MANY, 1))))
horiz_batch = np.hstack((horizontal, np.tile(weights, (HOW_MANY, 1))))

In [16]:
x_1 = np.vstack((vert_batch[0], horiz_batch[0]))
x_2 = np.vstack((vert_batch[1], horiz_batch[1]))
len(vert_batch[0])
x_2.max(), x_2.min()

(1.0, -0.9988916748869596)

### Global Sensitivity - 1st Try

In [19]:
# global sensitivity
# global_sens =  max_{x, x′} (2 * ‖G_x1‖₂)

N = grad_norm(x_1) * 2
#N, norm_func(x_1) 
# bounds = [(0,1), (0, 1)]
bounds =[(None, None), ]*2

sol = shgo(norm_func, bounds ) #, ((1,2), (0.5,3)))

# # SHGO minimises the function given certain constraints, in this case the bounds on the variables.
# sol = shgo(norm_func, ((1,2), (0.5,3)))
# -sol.fun

ValueError: Broadcasting failed (input/arg size mismatch)

In [91]:
# global sensitivity
# max_{x, x′} ‖G_x1 - G_x2‖₂

full_grad_func
norm = np.linalg.norm((G_x1 - G_x2), ord=2)
norm

0.005152726515461362

In [92]:
# LR = 0.1

# HOW_MANY = 1000

# vertical = get_vertical(HOW_MANY)
# horizontal = get_horizontal(HOW_MANY)

# weights = np.random.uniform(-1., 1., len(np.concatenate((w1.flatten(), b.flatten(), w2.flatten()))))[None, ...]
# vert_batch = np.hstack((vertical, np.tile(weights, (HOW_MANY, 1))))
# horiz_batch = np.hstack((horizontal, np.tile(weights, (HOW_MANY, 1))))
    
# # for i in range(1000):
# #     if i%100 ==0: print(loss_func(np.vstack((vert_batch, horiz_batch))).mean())
# #     G = grad_func(np.vstack((vert_batch,horiz_batch))).mean(axis=0)
# #     weights = weights - (G*LR)
# #     vert_batch = np.hstack((vertical, np.tile(weights, (HOW_MANY, 1))))
# #     horiz_batch = np.hstack((horizontal, np.tile(weights, (HOW_MANY, 1))))